In [1]:
!pip install neomodel
!pip install pandas

In [1]:
from DataSetupPackage import DataSetUpPackage
from DataAccessLayer import DataAccessLayer
import json

In [2]:
dbcon = DataAccessLayer(username='neo4j',password='neo123456').CreateDBConnection
dbcon.ClearCurrentDB  # This will clean up the neo4jDB
DataSetUpPackage().ReadCSVAndPopulateDB.PopulateNeo4j(dbcon.Session, verbose=True)

create Disease node with name as Fin or tail rot
Set props on Disease-Fin or tail rot:
environment - ALL
affectfish - nan
cause - The most common causes of fin rot are poor water quality and improperly-low water temperature. 
Overcrowding the tank, feeding outdated food, overfeeding the fish, and moving or handling, bullying from other fish can also cause stress that leads to fin rot.
treatment - Several antibiotics are effective in treating fin rot, but the root cause must be addressed to ensure the disease doesnât return.
Treatment should include a water change and careful examination of the aquarium conditions. 
If there is food debris, vacuum the gravel and take care to avoid overfeeding in the future.
Start putting dates on your fish food, as it loses the vitamin content fairly quickly after the food container is opened. 
Feeding your fish fresh, high-quality food in smaller quantities is far better than frequent, large feedings of stale foods.
Check the pH and temperature of th

C:\Anaconda3\envs\MRRSProject\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


create Symptom node with name as WHITEEDGE
create Fin or tail rot -hasSymptom-> WHITEEDGE
create WHITEEDGE -isDetectedIn-> Fin or tail rot
create Symptom node with name as FRAYFIN
create Fin or tail rot -hasSymptom-> FRAYFIN
create FRAYFIN -isDetectedIn-> Fin or tail rot
create Symptom node with name as FRAYTAIL
create Fin or tail rot -hasSymptom-> FRAYTAIL
create FRAYTAIL -isDetectedIn-> Fin or tail rot
create Symptom node with name as SWIMDIFF
create Fin or tail rot -hasSymptom-> SWIMDIFF
create SWIMDIFF -isDetectedIn-> Fin or tail rot
create Symptom node with name as LOSSAPPETITE
create Fin or tail rot -hasSymptom-> LOSSAPPETITE
create LOSSAPPETITE -isDetectedIn-> Fin or tail rot
create Symptom node with name as SETTLEBOTTOM
create Fin or tail rot -hasSymptom-> SETTLEBOTTOM
create SETTLEBOTTOM -isDetectedIn-> Fin or tail rot
create Medication node with name as ANTIBIOTICSgnveORG
create Fin or tail rot -useMedication-> ANTIBIOTICSgnveORG
create ANTIBIOTICSgnveORG -knownTreatment-> Fi

### Visualize created knowledge graph at: http://localhost:7474/browser/

* Username: neo4j

* Password: ai-user

In [3]:
dbcon = DataAccessLayer(username='neo4j',password='neo123456').CreateDBConnection
result2 = dbcon.GetAllNodeListOfType('Disease')
for resulti in result2:
    print("Disease Name: " + resulti.name)
    print(resulti.environment)
    for r in resulti.symptoms:
        print(r.name + " # " +str(r.id) +" === " + str(resulti.symptoms.relationship(r).weight) + str(resulti.symptoms.relationship(r).penalty))
    print("----------------------------")

Disease Name: Columnaris
Freshwater
COTTONGROWTH # 23 === 1.0-0.2
GREYMOUTH # 22 === 1.0-0.2
LOSSAPPETITE # 233 === 1.0-0.2
RAPIDBREATH # 21 === 1.0-0.2
BROWNGILLS # 20 === 1.0-0.2
MUCUSHEAD # 19 === 1.0-0.2
GREYGILL # 18 === 1.0-0.2
SKINPEEL # 17 === 1.0-0.2
SKINULCER # 14 === 1.0-0.2
FRAYFIN # 197 === 1.0-0.2
----------------------------
Disease Name: Dropsy
ALL
RAPIDGILL # 34 === 1.0-0.2
LOSSAPPETITE # 233 === 1.0-0.2
LETHARGIC # 33 === 1.0-0.2
EYEPROD # 32 === 1.0-0.2
SCALEOUT # 31 === 1.0-0.2
BLOAT # 30 === 1.0-0.2
----------------------------
Disease Name: Fish Tuberculosis
ALL
HIDING # 42 === 1.0-0.2
LOSSAPPETITE # 233 === 1.0-0.2
YELLOWNODULE # 41 === 1.0-0.2
FINSCALELOSS # 40 === 1.0-0.2
FRAYFIN # 197 === 1.0-0.2
SKINULCER # 14 === 1.0-0.2
PALESKIN # 39 === 1.0-0.2
HOLLOWBELLY # 38 === 1.0-0.2
LETHARGIC # 33 === 1.0-0.2
----------------------------
Disease Name: Viral Hemorrhagic Septicemia 
ALL
PALEGILL # 50 === 1.0-0.2
ERSWIM # 49 === 1.0-0.2
GASPING # 48 === 1.0-0.2
RAPIDBR

In [4]:
newCase = dbcon.CreateOrGetCaseNode('case1')
print(newCase.name + " # " + str(newCase.id))

case1 # 215


In [5]:
symptom1 = dbcon.GetOneSymptomNode("WORMBODY")
print(symptom1.id)

86


In [6]:
dbcon = DataAccessLayer(username='neo4j',password='ai-user').CreateDBConnection
newCase = dbcon.CreateOrGetCaseNode('case1',None,True)
symptom1 = dbcon.GetOneSymptomNode("WORMBODY")
newCase = dbcon.UpdateSymptomToCase(newCase, symptom1, 0.5)
print(len(newCase.suspected_symptoms))
for r in newCase.suspected_symptoms:
    print(r.name + " # " +str(r.id) +" === " + str(newCase.suspected_symptoms.relationship(r).suspectedLevel))

1
WORMBODY # 86 === 0.5


In [7]:
for d in symptom1.diseases:
    dbcon.UpdateDiseaseToCase(newCase, d, 1/len(d.symptoms))
    print(d.name)

Anchor Worms


In [8]:
symptom2 = dbcon.GetOneSymptomNode("GASPING")
newCase = dbcon.UpdateSymptomToCase(newCase, symptom2)
for d in symptom2.diseases:
    confidence = 1/len(d.symptoms)
    rel = newCase.suspected_diseases.relationship(d)
    if rel:
        confidence += rel.confidence
    dbcon.UpdateDiseaseToCase(newCase, d, confidence)
    print(d.name + " # " + str(confidence))

Gill Mite # 0.16666666666666666
CO2 Poisoning # 0.3333333333333333
Ammonia Poisoning # 0.5
Swim Bladder Disease # 0.125
Viral Hemorrhagic Septicemia  # 0.1111111111111111


In [9]:
symptom2

<Symptom: {'name': 'GASPING', 'description': 'Gasping at the surface', 'type': 'Behaviourial', 'tm_token': ['gasp', 'surface'], 'category1': 'Breathing', 'category2': 'Movement', 'category3': 'nan', 'imageurl': 'nan', 'question': 'Is your fish gasping for air at the water surface?', 'id': 48}>